In [1]:
from typing import Tuple
import pandas as pd
from tqdm import tqdm
import os
import time
import random
import pickle
import numpy as np
from os.path import isfile, join
from pydub import AudioSegment
from scipy.io import wavfile

import torch
from torch import nn
from torchaudio import transforms

from models.audio_LSTMCNN import AudioLSTMCNN

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Constants

In [2]:
RATE = 44100
MEL_SPECTROGRAM_BUCKETS = 128
MEL_SPECTROGRAM_WINDOW_LENGTH = 224
SPECTROS_PER_SECOND = RATE // (MEL_SPECTROGRAM_WINDOW_LENGTH / 2) - 1
CHUNKS_PER_SECOND = 2
CHUNK_SIZE_IN_SPECTROS = int(SPECTROS_PER_SECOND // CHUNKS_PER_SECOND)

TRAINSET_RATIO = 0.8

<hr>

# Dataset

## Create dataset

#### Load data

In [3]:
AUDIO_FOLDER = "C:\\Users\\amity\\PycharmProjects\\aimpathy\\data\\PMEmo\\PMEmo2019\\chorus"
THAYER_ANOTATIONS_CSV = "C:\\Users\\amity\\PycharmProjects\\aimpathy\\data\\PMEmo\\PMEmo2019\\annotations\\dynamic_annotations.csv"
thayer_annotations_df = pd.read_csv(THAYER_ANOTATIONS_CSV)

In [4]:
audio_files = [f for f in os.listdir(AUDIO_FOLDER) if isfile(join(AUDIO_FOLDER, f))]
audio_data = dict()
torch_spectorgrams = dict()
spectorgrammer = transforms.MelSpectrogram(sample_rate=RATE, n_fft=(MEL_SPECTROGRAM_BUCKETS * 2 - 2), win_length=MEL_SPECTROGRAM_WINDOW_LENGTH, power=2, normalized=True)

C:\Users\amity\Anaconda3\envs\aimpathy\lib\site-packages\torchaudio\functional\functional.py:539: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (128) may be set too low.
  warnings.warn(


#### Convery to Spectrograms

In [5]:
spectrograms = dict()
for audio_file in tqdm(audio_files, total=len(audio_files)):
    sound = AudioSegment.from_mp3(os.sep.join([AUDIO_FOLDER, audio_file])).set_channels(1)
    audio_file_wave = sound.export(format="wav", bitrate=RATE)
    sample_rate, samples = wavfile.read(audio_file_wave)
    spectogram = spectorgrammer(torch.from_numpy(samples/(2**15)).float().reshape((1, -1)))
    spectrograms[audio_file] = spectogram
    audio_file_wave.close()
spectrograms = {key: value for key, value in spectrograms.items() if ".wav" not in key}

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 794/794 [03:12<00:00,  4.13it/s]


#### Create final dataset

In [6]:
dataset = dict()  # music_id: spectrogram, (valence, arousal)
for file_name, spectrogram in tqdm(spectrograms.items(), total=len(spectrograms)):
    music_id = int(file_name.replace(".mp3", ""))
    dataset[music_id] = list()
    max_frame_time = thayer_annotations_df[thayer_annotations_df["musicId"] == music_id]["frameTime"].max()
    if np.isnan(max_frame_time):
        # print(f"Skipping {music_id}")
        continue
    for i in range(int(CHUNKS_PER_SECOND * max_frame_time)):
        data_df = thayer_annotations_df[(thayer_annotations_df["musicId"] == music_id) & (thayer_annotations_df["frameTime"] == i/2)]
        if data_df.empty:
            #  print(f"Skipping {musicI_id} - {i/2}")
            continue
        valence = data_df.iloc[0]["Valence(mean)"]
        arousal = data_df.iloc[0]["Arousal(mean)"]
        dataset[music_id].append((spectrogram[0, :, int((i/2-1)*CHUNK_SIZE_IN_SPECTROS): int((i/2)*CHUNK_SIZE_IN_SPECTROS)], (valence, arousal)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 793/793 [00:33<00:00, 23.93it/s]


### Divide to train and test

In [ ]:
train_music_ids = random.sample(dataset.keys(), int(TRAINSET_RATIO*len(dataset)))
trainset = {key: value for key, value in dataset.items() if key in train_music_ids}
testset = {key: value for key, value in dataset.items() if key not in train_music_ids}

In [ ]:
with open(f'data/datasets/trainset_{int(time.time())}.pk', 'wb') as f:
    pickle.dump(list(trainset.keys()), f)
with open(f'data/datasets/testset_{int(time.time())}.pk', 'wb') as f:
    pickle.dump(list(testset.keys()), f)

### Load train and test

In [7]:
dataset_label = '1674741184'
with open(f'data/datasets/trainset_{dataset_label}.pk', 'rb') as f:
    trainset_ids = pickle.load(f)
    trainset = {key: value for key, value in dataset.items() if key in trainset_ids}
with open(f'data/datasets/testset_{dataset_label}.pk', 'rb') as f:
    testset_ids = pickle.load(f)
    testset = {key: value for key, value in dataset.items() if key in testset_ids}

## Plot spectrogram

In [ ]:
import matplotlib.pyplot as plt
import librosa

In [ ]:
def plot_spectrogram(spec, title=None, ylabel="freq_bin", aspect="auto", xmax=None, ymax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(spec), origin="lower", aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    if ymax:
        axs.set_ylim((0, ymax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [ ]:
plot_spectrogram(dataset[1000][0], title="MelSpectrogram - torchaudio", ylabel="mel freq")

# Model

In [ ]:
# Load model
# model_name = f'saved_models/{}.pt'
# model_c = AudioLSTMCNN(input_shape=(128, 196), out_size=2).cuda()
# model_c.load_state_dict(torch.load(latest_model_name)).cuda();
# model_c.eval()
# print("Loaded model")

### Train model

In [8]:
model_c = AudioLSTMCNN(input_shape=(128, 196), out_size=2).cuda()

In [9]:
criterion = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model_c.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5, factor=0.5)

In [10]:
print(dataset[10][0][0].shape)
print(dataset[10][0][0].reshape((1, 1, MEL_SPECTROGRAM_BUCKETS, -1)).shape)

torch.Size([128, 196])
torch.Size([1, 1, 128, 196])


In [17]:
start_time = time.time()
EPOCS = 100
PRINT_MARK = 500
BATCH_SIZE = 50
STOP_LOSS = 0.005
MIN_LEARNING_RATE = 0.001

normalize = lambda x: (x + 1)/2
denormalize = lambda x: x*2 - 1


for epoc in range(EPOCS):
    losses = list()
    train_key_sample = random.sample(trainset.keys(), BATCH_SIZE)
    train_sample = [datum for key, value in trainset.items() if key in train_key_sample for datum in value]
    for batch_i, (X_train, (valence, arousal)) in enumerate(train_sample):
        optimizer.zero_grad()
        model_c.hidden = (torch.zeros(model_c.hidden[0].shape).cuda(),
                        torch.zeros(model_c.hidden[0].shape).cuda())
        
        y_train = torch.Tensor((valence, arousal)).cuda()
        # Apply the model
        y_pred = model_c(X_train.cuda())  # we don't flatten X-train here
        loss = criterion(y_pred, y_train)

        # Update parameters
        loss.backward(retain_graph=True)
        optimizer.step()

        losses.append(loss.cpu().item())
        
        # Print interim results
        if (batch_i > 0 or epoc == 0) and batch_i%PRINT_MARK == 0:
            print(f'{epoc:2}-{batch_i:4} | loss: {loss.item():.5f}[{np.mean(losses):.5f}] [{abs(y_train[0] - y_pred[0]):.5f}, {abs(y_train[1] - y_pred[1]):.5f}]        lr: {optimizer.param_groups[0]["lr"]}\
                  {y_train[0].item():.5f}, {y_train[0].item():.5f}  |  {y_pred[0].item():.5f}, {y_pred[0].item():.5f}')
    
    scheduler.step(np.mean(losses))

    if np.mean(losses) < STOP_LOSS or optimizer.param_groups[0]["lr"] < MIN_LEARNING_RATE:
        break
        
latest_model_name = f'saved_models/AudioLSTMCNN_{int(time.time())}_{str(np.mean(losses))[2:]}.pt'
torch.save(model_c.state_dict(), latest_model_name)
print(f"Saved model {latest_model_name}")
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

 0-   0 | loss: 0.09216[0.09216] [0.30165, 0.30548]        lr: 0.001                  0.42855, 0.42855  |  0.73020, 0.73020
 0- 500 | loss: 0.00329[0.00964] [0.02967, 0.07552]        lr: 0.001                  0.67066, 0.67066  |  0.64100, 0.64100
 0-1000 | loss: 0.00026[0.01013] [0.00441, 0.02216]        lr: 0.001                  0.73422, 0.73422  |  0.72982, 0.72982
 0-1500 | loss: 0.00100[0.01047] [0.00571, 0.04445]        lr: 0.001                  0.82501, 0.82501  |  0.83072, 0.83072
 0-2000 | loss: 0.00511[0.01018] [0.06974, 0.07313]        lr: 0.001                  0.55889, 0.55889  |  0.62862, 0.62862
 1- 500 | loss: 0.00476[0.00576] [0.06456, 0.07315]        lr: 0.001                  0.55811, 0.55811  |  0.62267, 0.62267
 1-1000 | loss: 0.00413[0.00891] [0.06555, 0.06297]        lr: 0.001                  0.40189, 0.40189  |  0.46743, 0.46743
 1-1500 | loss: 0.00016[0.01032] [0.01615, 0.00752]        lr: 0.001                  0.75956, 0.75956  |  0.77571, 0.77571
 2- 500 

16-2000 | loss: 0.00802[0.02713] [0.12263, 0.03148]        lr: 0.00025                  0.49246, 0.49246  |  0.61510, 0.61510
17- 500 | loss: 0.00851[0.02547] [0.06935, 0.11049]        lr: 0.00025                  0.72601, 0.72601  |  0.65666, 0.65666
17-1000 | loss: 0.05897[0.03389] [0.23344, 0.25186]        lr: 0.00025                  0.82655, 0.82655  |  0.59311, 0.59311
17-1500 | loss: 0.03702[0.03418] [0.14134, 0.23254]        lr: 0.00025                  0.20368, 0.20368  |  0.34501, 0.34501
17-2000 | loss: 0.00304[0.03131] [0.07788, 0.00437]        lr: 0.00025                  0.71024, 0.71024  |  0.63236, 0.63236
18- 500 | loss: 0.00302[0.01822] [0.02098, 0.07478]        lr: 0.000125                  0.73200, 0.73200  |  0.71102, 0.71102
18-1000 | loss: 0.00913[0.01646] [0.04594, 0.12709]        lr: 0.000125                  0.60744, 0.60744  |  0.65338, 0.65338
18-1500 | loss: 0.00271[0.01735] [0.02336, 0.06984]        lr: 0.000125                  0.67623, 0.67623  |  0.6528

### Test model

In [18]:
losses = list()

with torch.no_grad():
    test_sample = [datum for key, value in testset.items() for datum in value]
    for batch_i, (X_test, (valence, arousal)) in tqdm(enumerate(test_sample), total=len(test_sample)):
        model_c.hidden = (torch.zeros(model_c.hidden[0].shape).cuda(),
                        torch.zeros(model_c.hidden[0].shape).cuda())
        y_test = torch.Tensor((valence, arousal)).cuda()
        # Apply the model
        y_val = model_c(X_test.cuda())
        loss = criterion(y_val, y_test)
        losses.append(loss.cpu())
        if (min(y_val).item() < 0.5):
            print(y_val)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 6635/6635 [00:27<00:00, 241.40it/s]


In [19]:
print(f"Mean: {np.mean(losses):.5f}, median: {np.median(losses):.5f}")

Mean: 0.03146, median: 0.02223


In [16]:
print(f"Mean: {np.mean(losses):.5f}, median: {np.median(losses):.5f}")

Mean: 0.06930, median: 0.03384


#### Inspect quadrants

In [ ]:
testset.keys()

In [ ]:
test_index = 519
test_location = 0

with torch.no_grad():
    pred = model_c(testset[test_index][test_location][0].cuda()).cpu()
print(pred)
print([denormalize(a.item()) for a in pred])
print(testset[test_index][test_location][1])

In [ ]:
[(key, value[0][1]) for key, value in testset.items() if value and value[0][1] < (0.5, 0.5)]